In [1]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import pennylane as qml

from sklearn.model_selection import train_test_split

# Direct classes included for easyness
from tensorflow.keras.utils import plot_model
from tensorflow.keras.backend import clear_session
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.backend import clear_session
from tensorflow.keras.models import Model

In [2]:
# load data
np_data = np.load("data/dataRS.npy")

noise_output = np_data[:,-1].reshape(len(np_data),1)
input = np_data[:,:-1]

# split data
test_split = 0.33
x_train, x_test, y_train, y_test = train_test_split(input, noise_output, test_size=test_split)

In [3]:
t3 = tf.constant([[1, 3, 5, 7, 4],
                   [9, 11, 13, 15, 44],
                   [17, 19, 21, 23, 33],
                   [25, 27, 29, 31, 22]
                  ])
print(t3[0])


tf.Tensor([1 3 5 7 4], shape=(5,), dtype=int32)


In [10]:
from pennylane import numpy as np
# In a first try, we use as much qubits as we have input params
global n_qubits
encoding_dim = 2
inDim = x_train.shape[1]
n_qubits = encoding_dim
n_layer = 2
weight_shapes = {"weights": (n_layer,n_qubits,3)}
n_qcParams = n_layer*3*n_qubits

dev = qml.device("default.qubit", wires=n_qubits)  # number of shots default? =1 du Horst

@qml.qnode(dev)
def qnode(inputs, weights):
    freq = input[:,0]#tf.gather(inputs, [0], axis=1)
    chor = input[:,1]#tf.gather(inputs, [1], axis=1)
    angl = input[:,2]#tf.gather(inputs, [2], axis=1)
    velo = input[:,3]#tf.gather(inputs, [3], axis=1)
    ssdt = input[:,4]#tf.gather(inputs, [4], axis=1)
    # Bring everything in rotation of the two qubits
    qml.Rot(freq, chor, angl, wires=0)
    qml.Rot(velo, ssdt, angl, wires=1)
    qml.Hadamard(wires=0)
    qml.Hadamard(wires=1)
    qml.StronglyEntanglingLayers(weights, wires=range(n_qubits))
    qml.Hadamard(wires=0)
    qml.Hadamard(wires=1)
    return [qml.expval(qml.PauliZ(wires=i)) for i in range(n_qubits)]
qlayer = qml.qnn.KerasLayer(qnode, weight_shapes, output_dim=n_qubits)
print(x_train[0,:])
print(qml.draw(qnode, expansion_strategy="device")(x_train[:,0], np.random.rand(n_layer,n_qubits,3)))

[0.01515152 0.18018018 0.72727273 0.         0.08084723]
0: ──Rot──H──Rot(0.48,0.50,0.19)─╭●─╭X──Rot(0.41,0.84,0.92)─╭●─╭X──H─┤  <Z>
1: ──Rot──H──Rot(0.88,0.08,0.28)─╰X─╰●──Rot(0.30,0.59,0.65)─╰X─╰●──H─┤  <Z>


In [ ]:
dev = qml.device("default.qubit", wires=n_qubits)

@qml.encoder(dev)
def encoder(inputs, weights):
    
    return [qml.expval(qml.PauliZ(wires=i)) for i in range(n_qubits)]

In [11]:
#Create a autoencode like in https://blog.keras.io/building-autoencoders-in-keras.html
clear_session()

inputLayer = Input(shape=(x_train.shape[1],))
encoded = qlayer(inputLayer)
translate = Dense(n_qubits,activation='linear')(encoded)
decoded = Dense(x_train.shape[1], activation="relu")(translate)
autoencoder = Model(inputLayer, decoded)
print(f"Params in the quantum layer to train: {n_qcParams}")
autoencoder.summary()

plot_model(autoencoder)

Params in the quantum layer to train: 12
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 5)]               0         
                                                                 
 keras_layer (KerasLayer)    (None, 2)                 0 (unused)
                                                                 
 dense (Dense)               (None, 2)                 6         
                                                                 
 dense_1 (Dense)             (None, 5)                 15        
                                                                 
Total params: 21
Trainable params: 21
Non-trainable params: 0
_________________________________________________________________
You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) for plot_model to work.


In [6]:
# Encoder
encoder = Model(inputLayer, encoded)
# Decoder
encoded_input = Input(shape=(encoding_dim,))
translate_layer = autoencoder.layers[-2]
decoder_layer = autoencoder.layers[-1]
decoder = Model(encoded_input, decoder_layer(translate_layer(encoded_input)))


In [7]:
#Training
#['mean_squared_error', 'mean_absolute_error','mean_squared_logarithmic_error', 'mean_absolute_percentage_error']
autoencoder.compile(optimizer = 'rmsprop', loss='mean_squared_error', metrics=['mean_squared_error', 'mean_absolute_error'])
autoencoder_history = autoencoder.fit(x_train, x_train, epochs=2, batch_size=25, validation_split=0.25)

Epoch 1/2


InvalidArgumentError: Exception encountered when calling layer 'keras_layer' (type KerasLayer).

{{function_node __wrapped__GatherV2_device_/job:localhost/replica:0/task:0/device:CPU:0}} Shape must be at least rank 2 but is rank 1 [Op:GatherV2]

Call arguments received by layer 'keras_layer' (type KerasLayer):
  • inputs=tf.Tensor(shape=(25, 5), dtype=float32)

In [ ]:
def plot_metrics(history):
    n = len(history.history.keys())//2
    fig,axs = plt.subplots(1,n, figsize=(18,5))

    for i,[key,val] in enumerate(history.history.items()):
        axs[i%n].plot(history.history[key], lw=4, label=key.replace("_", " "))
    
    for ax in axs:
        #ax.set_yscale("log")
        ax.set_xlabel("epoch", fontsize=16)
        ax.legend(fontsize=14)
plot_metrics(autoencoder_history)
plt.savefig("archive/history-Qencoder-16032023.png")

In [ ]:
evaluation = autoencoder.evaluate(x_test[:-1], x_test[:-1])
encoded_test = encoder.predict(x_test)
# decoded_test = decoder.predict(encoded_test)
print(f'Data: {x_test[0,:]} to be: {encoded_test[0,:]} and rebuilded it to: {decoded_test[0,:]}')

In [ ]:
encoder.save("archive/model-Qencoder-16032023.h5")
config = encoder.get_config()
np.save("archive/config-Qencoder-16032023.npy", config)